In [1]:
import pandas as pd
import numpy as np

pd.set_option("mode.copy_on_write", True)
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [2]:
data = pd.read_csv("train_data.csv")
data.head()

,game_id,date,Team,Opponent,Team_Score,Opponent_Score,Location,ADJO,ADJD,EFF,EFG_pct,TO_Pct,OR_Pct,FTR_Pct,ThreePointer_Pct,opp_ADJO,opp_ADJD,Opp_EFF,Opp_EFG_Pct,Opp_TO_Pct,Opp_OR_Pct,Opp_FTR_Pct,Opp_ThreePointer_Pct
0,Abilene ChristianBaylor12-9,2024-12-09,Abilene Christian,Baylor,57,88,A,94.8,112.7,86.7,36.4,15.2,27.0,41.8,14.29,130.2,93.9,133.8,66.1,13.7,35.7,27.1,44.44
1,Abilene ChristianKennesaw St.11-20,2024-11-20,Abilene Christian,Kennesaw St.,78,84,A,108.4,115.4,107.1,50.0,9.6,20.0,50.8,33.33,112.7,114.4,115.3,62.2,26.1,31.8,100.0,43.75
2,Abilene ChristianMontana St.11-26,2024-11-26,Abilene Christian,Montana St.,59,85,A,90.6,129.1,91.6,44.0,12.4,20.6,20.7,16.67,131.3,95.8,131.9,75.0,20.2,25.0,26.0,65.38
3,Abilene ChristianNebraska Omaha11-30,2024-11-30,Abilene Christian,Nebraska Omaha,71,55,A,105.4,84.1,107.2,51.9,15.1,24.1,35.2,30.77,81.2,114.1,83.1,42.0,24.2,17.2,54.5,31.25
4,Abilene ChristianNew Mexico St.12-4,2024-12-04,Abilene Christian,New Mexico St.,78,70,A,117.5,106.9,117.8,58.9,16.6,32.0,77.8,43.75,105.8,123.0,105.7,51.0,18.1,27.6,58.0,33.33


In [3]:
data.dtypes

game_id                  object
date                     object
Team                     object
Opponent                 object
Team_Score                int64
Opponent_Score            int64
Location                 object
ADJO                    float64
ADJD                    float64
EFF                     float64
EFG_pct                 float64
TO_Pct                  float64
OR_Pct                  float64
FTR_Pct                 float64
ThreePointer_Pct        float64
opp_ADJO                float64
opp_ADJD                float64
Opp_EFF                 float64
Opp_EFG_Pct             float64
Opp_TO_Pct              float64
Opp_OR_Pct              float64
Opp_FTR_Pct             float64
Opp_ThreePointer_Pct    float64
dtype: object

In [4]:
data.isnull().sum()

game_id                 0
date                    0
Team                    0
Opponent                0
Team_Score              0
Opponent_Score          0
Location                0
ADJO                    0
ADJD                    0
EFF                     0
EFG_pct                 0
TO_Pct                  0
OR_Pct                  0
FTR_Pct                 0
ThreePointer_Pct        0
opp_ADJO                0
opp_ADJD                0
Opp_EFF                 0
Opp_EFG_Pct             0
Opp_TO_Pct              0
Opp_OR_Pct              0
Opp_FTR_Pct             0
Opp_ThreePointer_Pct    0
dtype: int64

In [5]:
data[data.isnull().any(axis=1)]

,game_id,date,Team,Opponent,Team_Score,Opponent_Score,Location,ADJO,ADJD,EFF,EFG_pct,TO_Pct,OR_Pct,FTR_Pct,ThreePointer_Pct,opp_ADJO,opp_ADJD,Opp_EFF,Opp_EFG_Pct,Opp_TO_Pct,Opp_OR_Pct,Opp_FTR_Pct,Opp_ThreePointer_Pct


In [6]:
def preprocess(data):
    data_clean = data.dropna()
    columns_to_convert = [
        "ADJO",
        "ADJD",
        "EFG_pct",
        "TO_Pct",
        "OR_Pct",
        "FTR_pct",
        "ThreePointer_Pct",
        "opp_ADJO",
        "opp_ADJD",
        "Opp_EFG_Pct",
        "Opp_TO_Pct",
        "Opp_OR_Pct",
        "Opp_FTR_Pct",
        "Opp_ThreePointer_Pct",
    ]
    categorical_columns = ["Location"]
    data_encoded = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

    data_encoded = data_encoded.apply(
        lambda x: x.astype(int) if x.dtype == "bool" else x
    )

    return data_encoded

In [7]:
preprocessed_data = preprocess(data)

In [8]:
X = preprocessed_data.drop(
    columns=[
        "Team_Score",
        "Opponent_Score",
        "Team",
        "Opponent",
        "date",
        "game_id",
        "Opp_EFF",
        "EFF",
    ]
)

y = preprocessed_data[["Team_Score", "Opponent_Score"]]

X = X.fillna(0)

In [9]:
X.dtypes

ADJO                    float64
ADJD                    float64
EFG_pct                 float64
TO_Pct                  float64
OR_Pct                  float64
FTR_Pct                 float64
ThreePointer_Pct        float64
opp_ADJO                float64
opp_ADJD                float64
Opp_EFG_Pct             float64
Opp_TO_Pct              float64
Opp_OR_Pct              float64
Opp_FTR_Pct             float64
Opp_ThreePointer_Pct    float64
Location_N                int64
dtype: object

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [11]:
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

In [12]:
model = MLPRegressor(
    hidden_layer_sizes=(32, 16), activation="relu", max_iter=500, random_state=42
)

In [13]:
model.fit(X_train, y_train)

MLPRegressor(hidden_layer_sizes=(32, 16), max_iter=500, random_state=42)

In [14]:
y_pred_scaled = model.predict(X_test)
y_pred = scaler_y.inverse_transform(y_pred_scaled)

In [15]:
# Ensure that y_test is 2D
y_test = np.array(y_test)

# Check if y_test and y_pred have matching shapes
print(f"Shape of y_test: {y_test.shape}")
print(f"Shape of y_pred: {y_pred.shape}")

# If y_test is 2D (it should be with shape (61, 2)), proceed with the metrics
if y_test.ndim == 2 and y_pred.ndim == 2:
    mae_team = mean_absolute_error(y_test[:, 0], y_pred[:, 0])
    mae_opponent = mean_absolute_error(y_test[:, 1], y_pred[:, 1])

    mse_team = mean_squared_error(y_test[:, 0], y_pred[:, 0])
    mse_opponent = mean_squared_error(y_test[:, 1], y_pred[:, 1])

    r2_team = r2_score(y_test[:, 0], y_pred[:, 0])
    r2_opponent = r2_score(y_test[:, 1], y_pred[:, 1])

    print("Team Score Prediction Metrics:")
    print(f"MAE: {mae_team}")
    print(f"MSE: {mse_team}")
    print(f"R² Score: {r2_team}")

    print("\nOpponent Score Prediction Metrics:")
    print(f"MAE: {mae_opponent}")
    print(f"MSE: {mse_opponent}")
    print(f"R² Score: {r2_opponent}")
else:
    print("The shapes of y_test and y_pred do not match.")

Shape of y_test: (405, 2)
Shape of y_pred: (405, 2)
Team Score Prediction Metrics:
MAE: 68.05719373063073
MSE: 4743.463168202311
R² Score: -3979.1974207960675

Opponent Score Prediction Metrics:
MAE: 76.938146203868
MSE: 6025.734949566739
R² Score: -5356.7251088513


### Using Cross Validation

In [16]:
cv_model = MLPRegressor(
    hidden_layer_sizes=(64, 32), activation="relu", max_iter=500, random_state=42
)

In [17]:
# Team score
y_train_team_cv = y_train[:, 0]
# Opponent Score
y_train_opponent_cv = y_train[:, 1]

In [18]:
# Perform 5-fold cross-validation for Team_score
cv_scores_team = cross_val_score(
    model, X_train, y_train_team_cv, cv=5, scoring="neg_mean_absolute_error"
)

# Perform 5-fold cross-validation for Opponent_score
cv_scores_opponent = cross_val_score(
    model, X_train, y_train_opponent_cv, cv=5, scoring="neg_mean_absolute_error"
)

# Convert negative MAE scores to positive
cv_scores_team = -cv_scores_team
cv_scores_opponent = -cv_scores_opponent

# Print the cross-validation results
print(
    f"Cross-validated MAE for Team Score: {cv_scores_team.mean()} ± {cv_scores_team.std()}"
)
print(
    f"Cross-validated MAE for Opponent Score: {cv_scores_opponent.mean()} ± {cv_scores_opponent.std()}"
)

/Users/nrutachoudhari/opt/miniconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/nrutachoudhari/opt/miniconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/nrutachoudhari/opt/miniconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/nrutachoudhari/opt/miniconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged ye

Cross-validated MAE for Team Score: 0.4850933116172575 ± 0.030661165649031943
Cross-validated MAE for Opponent Score: 0.5267195238242192 ± 0.021237231084093436


In [19]:
# Fit the model on the full training data
cv_model.fit(X_train, y_train)

# Make predictions on the test data
cv_ypred_scaled = cv_model.predict(X_test)

# Inverse transform the predictions for both Team_score and Opponent_score
cv_y_pred = scaler_y.inverse_transform(cv_ypred_scaled)

# Evaluate the model's performance
cv_mae_team = mean_absolute_error(y_test[:, 0], cv_y_pred[:, 0])
cv_mae_opponent = mean_absolute_error(y_test[:, 1], cv_y_pred[:, 1])

cv_mse_team = mean_squared_error(y_test[:, 0], cv_y_pred[:, 0])
cv_mse_opponent = mean_squared_error(y_test[:, 1], cv_y_pred[:, 1])

cv_r2_team = r2_score(y_test[:, 0], cv_y_pred[:, 0])
cv_r2_opponent = r2_score(y_test[:, 1], cv_y_pred[:, 1])

# Print the evaluation metrics
print("\nTeam Score Prediction Metrics:")
print(f"MAE: {cv_mae_team}")
print(f"MSE: {cv_mse_team}")
print(f"R² Score: {cv_r2_team}")

print("\nOpponent Score Prediction Metrics:")
print(f"MAE: {cv_mae_opponent}")
print(f"MSE: {cv_mse_opponent}")
print(f"R² Score: {cv_r2_opponent}")


Team Score Prediction Metrics:
MAE: 68.21696338956342
MSE: 4787.893752311237
R² Score: -4016.478725615776

Opponent Score Prediction Metrics:
MAE: 76.83710731109869
MSE: 6036.346869387638
R² Score: -5366.1605967633


### Hyperparameter tuning on CV

In [20]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    "hidden_layer_sizes": [(32,), (64, 32), (128, 64, 32)],  # Different layer sizes
    "activation": ["relu", "tanh"],  # Activation functions
    "solver": ["adam", "sgd"],  # Solvers
    "alpha": [0.0001, 0.001, 0.01],  # L2 regularization
    "learning_rate_init": [0.001, 0.01],  # Learning rate
    "max_iter": [500],  # Maximum number of iterations
}

# Initialize the MLPRegressor
mlp = MLPRegressor(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=mlp,
    param_grid=param_grid,
    scoring="neg_mean_absolute_error",  # Use MAE as the evaluation metric
    cv=5,  # 5-fold cross-validation
    verbose=2,  # Display progress
    n_jobs=-1,  # Use all available CPU cores
)

# Perform grid search
grid_search.fit(X_train, y_train)

# Display the best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best CV Score (MAE):", -grid_search.best_score_)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(32,), learning_rate_init=0.001, max_iter=500, solver=sgd; total time=   0.3s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(32,), learning_rate_init=0.001, max_iter=500, solver=sgd; total time=   0.3s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(32,), learning_rate_init=0.001, max_iter=500, solver=sgd; total time=   0.3s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(32,), learning_rate_init=0.001, max_iter=500, solver=adam; total time=   0.4s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(32,), learning_rate_init=0.01, max_iter=500, solver=adam; total time=   0.2s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(32,), learning_rate_init=0.001, max_iter=500, solver=sgd; total time=   0.3s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(32,), learning_rate_init=0.001, max_iter=500, solver=ad

/Users/nrutachoudhari/opt/miniconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(32,), learning_rate_init=0.001, max_iter=500, solver=adam; total time=   0.9s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(32,), learning_rate_init=0.01, max_iter=500, solver=adam; total time=   0.3s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(32,), learning_rate_init=0.01, max_iter=500, solver=adam; total time=   0.3s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(32,), learning_rate_init=0.01, max_iter=500, solver=sgd; total time=   0.4s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(32,), learning_rate_init=0.01, max_iter=500, solver=sgd; total time=   0.7s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(32,), learning_rate_init=0.01, max_iter=500, solver=sgd; total time=   0.8s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(32,), learning_rate_init=0.01, max_iter=500, solver=sgd; total time=   0.8s
[CV] END activation=relu, alpha=0.0001, hidd

/Users/nrutachoudhari/opt/miniconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(64, 32), learning_rate_init=0.001, max_iter=500, solver=sgd; total time=   0.9s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(64, 32), learning_rate_init=0.001, max_iter=500, solver=sgd; total time=   1.0s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(64, 32), learning_rate_init=0.001, max_iter=500, solver=sgd; total time=   1.0s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(64, 32), learning_rate_init=0.001, max_iter=500, solver=adam; total time=   2.3s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(64, 32), learning_rate_init=0.01, max_iter=500, solver=adam; total time=   0.6s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(64, 32), learning_rate_init=0.001, max_iter=500, solver=adam; total time=   2.4s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(64, 32), learning_rate_init=0.001, max_iter=500, solver=adam; total time=   2.5s
[CV] END activati

...

[CV] END activation=tanh, alpha=0.01, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.01, max_iter=500, solver=sgd; total time=   6.1s
[CV] END activation=tanh, alpha=0.01, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.01, max_iter=500, solver=sgd; total time=   6.2s


/Users/nrutachoudhari/opt/miniconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/nrutachoudhari/opt/miniconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END activation=tanh, alpha=0.01, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.01, max_iter=500, solver=sgd; total time=   5.8s
[CV] END activation=tanh, alpha=0.01, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.01, max_iter=500, solver=sgd; total time=   5.6s


/Users/nrutachoudhari/opt/miniconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END activation=tanh, alpha=0.01, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.01, max_iter=500, solver=sgd; total time=   5.5s
Best Parameters: {'activation': 'tanh', 'alpha': 0.01, 'hidden_layer_sizes': (128, 64, 32), 'learning_rate_init': 0.001, 'max_iter': 500, 'solver': 'sgd'}
Best CV Score (MAE): 0.40247422131063476


In [26]:
# Train the best model
best_model = grid_search.best_estimator_  # or random_search.best_estimator_
best_model.fit(X_train, y_train)

# Predict on the test set
ypred_scaled = best_model.predict(X_test)
y_pred = scaler_y.inverse_transform(ypred_scaled)

# Evaluate performance
mae_team = mean_absolute_error(y_test[:, 0], y_pred[:, 0])
mae_opponent = mean_absolute_error(y_test[:, 1], y_pred[:, 1])

print(f"Test MAE for Team Score: {mae_team}")
print(f"Test MAE for Opponent Score: {mae_opponent}")

Test MAE for Team Score: 68.56400152379463
Test MAE for Opponent Score: 77.08317982654916


### Predict data

Games for the 28th of December, 2024

In [22]:
predict_data_28 = pd.read_csv("test_file_28th.csv")
predict_data_28

,date,Team,Opponent,Location,ADJO,ADJD,EFG_pct,TO_Pct,OR_Pct,FTR_Pct,ThreePointer_Pct,Opp_ADJO,Opp_ADJD,Opp_EFG_pct,Opp_TO_Pct,Opp_OR_Pct,Opp_FTR_Pct,Opp_ThreePointer_Pct
0,2024-12-28,Richmond,Florida Gulf Coast,H,101.758087,109.741049,48.3,15.0,20.7,40.6,29.2,105.710407,106.895812,51.6,17.2,25.6,18.1,33.1
1,2024-12-28,San Diego St.,Utah St.,H,110.402016,93.446262,51.3,16.0,30.4,26.3,36.1,116.010122,97.818219,53.5,17.3,39.5,38.1,31.6
2,2024-12-28,Georgia Tech,Alabama A&M,H,105.714147,101.885890,48.7,16.0,29.8,31.8,32.7,94.540931,118.331617,43.9,20.0,32.8,36.4,31.9
3,2024-12-28,San Jose St.,Boise St.,H,108.957965,108.642510,51.3,15.4,26.7,38.0,35.2,113.108156,97.762460,51.5,16.0,33.9,42.7,30.0
4,2024-12-28,Colorado St.,New Mexico,H,105.539411,100.325033,51.8,16.8,23.1,27.9,30.6,112.930069,100.274810,50.0,15.6,34.2,41.0,34.6
5,2024-12-28,UCLA,Gonzaga,N,109.930056,87.869252,54.3,16.4,34.2,34.9,35.8,124.118600,93.745664,55.7,13.5,32.9,32.9,34.8
6,2024-12-28,Wyoming,Nevada,H,104.934785,103.629449,52.9,21.1,34.5,36.9,34.8,115.399439,101.214327,57.4,15.9,29.7,38.8,41.6
7,2024-12-28,Texas A&M,Abilene Christian,H,112.021330,92.040856,47.8,18.4,44.5,39.8,31.6,100.344692,105.008776,48.3,19.1,28.6,43.6,27.2
8,2024-12-28,Louisville,Eastern Kentucky,H,114.541145,98.320567,51.2,17.1,33.9,37.1,29.8,110.376078,107.352351,48.5,15.4,33.5,22.1,32.0
9,2024-12-28,Maryland,Maryland Eastern Shore,H,117.891812,89.667452,57.3,13.4,33.9,29.2,37.1,99.924968,120.725914,43.8,19.2,22.0,35.3,30.0


In [23]:
# predict_data_28["Location"] = predict_data["Location"].replace(
#     {"Home": "H", "Away": "A", "Neutral": "N"}
# )

In [24]:
X.columns

Index(['ADJO', 'ADJD', 'EFG_pct', 'TO_Pct', 'OR_Pct', 'FTR_Pct',
       'ThreePointer_Pct', 'opp_ADJO', 'opp_ADJD', 'Opp_EFG_Pct', 'Opp_TO_Pct',
       'Opp_OR_Pct', 'Opp_FTR_Pct', 'Opp_ThreePointer_Pct', 'Location_N'],
      dtype='object')

In [27]:
predict_data_28 = predict_data_28.rename(
    columns={
        "Opp_ADJO": "opp_ADJO",
        "Opp_ADJD": "opp_ADJD",
        "Opp_EFG_pct": "Opp_EFG_Pct",
    }
)

In [28]:
predict_data_process_28 = preprocess(predict_data_28)
predict_data_process_28 = predict_data_process_28.drop(
    columns=["date", "Team", "Opponent"]
)
predict_data_process_28 = predict_data_process_28.fillna(0)
predict_data_scaled_28 = scaler_X.transform(predict_data_process_28)

In [29]:
# predicting using normal NN

nn_predictions_scaled_28 = model.predict(predict_data_scaled_28)
nn_predictions_28 = scaler_y.inverse_transform(nn_predictions_scaled_28)

# Add predictions to the original dataset
predict_data_28["NN_Team_Score"] = nn_predictions_28[:, 0]
predict_data_28["NN_Opp_Score"] = nn_predictions_28[:, 1]

In [30]:
# predicting using MLP with Cross Validation

cv_predictions_scaled_28 = cv_model.predict(predict_data_scaled_28)
cv_predictions_28 = scaler_y.inverse_transform(cv_predictions_scaled_28)

# Add predictions to the original dataset
predict_data_28["CV_Team_Score"] = cv_predictions_28[:, 0]
predict_data_28["CV_Opp_Score"] = cv_predictions_28[:, 1]

In [31]:
# predicting using CV with Hyperparameter Tuning

hpt_predictions_scaled_28 = best_model.predict(predict_data_scaled_28)
hpt_predictions_28 = scaler_y.inverse_transform(hpt_predictions_scaled_28)

# Add predictions to the original dataset
predict_data_28["HPT_Team_Score"] = hpt_predictions_28[:, 0]
predict_data_28["HPT_Opp_Score"] = hpt_predictions_28[:, 1]

In [29]:
# # adding the manual scores manually coz idk
# predict_data["Actual_Team"] = [89, 80, 80, 73, 81, 90, 83, 89]
# predict_data["Actual_Opp"] = [66, 77, 77, 79, 54, 46, 90, 58]

In [32]:
predict_data_28[
    [
        "Team",
        "Opponent",
        "NN_Team_Score",
        "NN_Opp_Score",
        "CV_Team_Score",
        "CV_Opp_Score",
        "HPT_Team_Score",
        "HPT_Opp_Score",
        # "Actual_Team",
        # "Actual_Opp",
    ]
]

,Team,Opponent,NN_Team_Score,NN_Opp_Score,CV_Team_Score,CV_Opp_Score,HPT_Team_Score,HPT_Opp_Score
0,Richmond,Florida Gulf Coast,74.509684,73.351950,67.497196,70.903107,70.160618,72.442188
1,San Diego St.,Utah St.,73.657590,78.507273,70.576115,73.887127,75.629494,81.744159
2,Georgia Tech,Alabama A&M,76.741118,68.738184,70.652008,64.000042,73.142715,63.030603
3,San Jose St.,Boise St.,77.934662,82.356320,73.837004,76.146803,76.699001,80.525638
4,Colorado St.,New Mexico,72.459916,76.502455,68.277148,71.399426,72.352756,77.993647
5,UCLA,Gonzaga,77.646361,74.963495,71.232008,75.584487,81.101754,82.805565
6,Wyoming,Nevada,71.599739,80.939263,73.068119,80.123635,75.587749,84.105606
7,Texas A&M,Abilene Christian,75.399888,67.025371,80.264551,70.975923,78.155661,67.302140
8,Louisville,Eastern Kentucky,76.150780,72.003252,78.322666,74.668876,77.394826,71.885199
9,Maryland,Maryland Eastern Shore,80.824721,60.335524,89.551437,66.438113,84.446689,59.334311


In [31]:
# predict_data[
#     [
#         "Team",
#         "Opponent",
#         "NN_Team_Score",
#         "NN_Opp_Score",
#         "CV_Team_Score",
#         "CV_Opp_Score",
#         "HPT_Team_Score",
#         "HPT_Opp_Score",
#         "Actual_Team",
#         "Actual_Opp",
#     ]
# ].to_csv("mlp_predictions.csv")

Games for the 29th of December, 2024

In [33]:
predict_data_29 = pd.read_csv("test_file_29th.csv")
predict_data_29

,date,Team,Opponent,Location,ADJO,ADJD,EFG_pct,TO_Pct,OR_Pct,FTR_Pct,ThreePointer_Pct,Opp_ADJO,Opp_ADJD,Opp_EFG_pct,Opp_TO_Pct,Opp_OR_Pct,Opp_FTR_Pct,Opp_ThreePointer_Pct
0,2024-12-29,Texas St.,UT Arlington,H,109.504793,104.248766,51.8,20.1,35.0,39.5,39.0,108.781738,110.471294,51.4,18.0,33.4,34.3,37.8
1,2024-12-29,Washington,NJIT,H,103.623921,100.980171,49.2,17.1,31.0,41.7,32.4,90.805077,110.393403,44.2,19.5,27.1,27.5,29.7
2,2024-12-29,North Carolina,Campbell,H,116.097482,99.425272,52.8,15.0,28.7,39.8,31.6,96.043679,108.132847,47.5,17.7,22.6,35.9,30.7
3,2024-12-29,Iona,Harvard,H,96.975713,106.396593,44.8,23.8,41.3,34.3,30.3,98.117396,109.223416,48.7,18.9,26.2,24.7,30.1
4,2024-12-29,Army,UTSA,H,104.290586,114.008871,49.4,14.0,28.6,26.3,31.9,104.279709,109.569165,48.3,18.8,30.4,29.8,34.0
5,2024-12-29,Northwestern,Northeastern,H,108.543184,92.738343,49.4,13.5,29.9,33.5,33.2,101.571496,101.445543,51.9,18.5,29.3,31.3,32.4
6,2024-12-29,Alabama,South Dakota St.,H,122.500118,95.996245,54.7,17.6,36.7,43.4,31.2,109.213988,102.139328,52.4,18.3,31.1,29.4,32.7
7,2024-12-29,Ohio St.,Indiana St.,H,114.330437,97.647266,57.4,15.3,28.1,35.2,39.9,109.379733,113.091958,59.1,21.2,26.5,29.6,38.0
8,2024-12-29,Illinois,Chicago St.,H,116.083343,90.659572,51.5,16.0,36.4,39.9,32.3,88.940352,110.366604,40.9,20.5,24.3,27.8,29.1
9,2024-12-29,Oregon,Weber St.,H,116.967225,95.369118,52.2,16.4,34.4,40.3,33.6,104.761323,111.655595,49.4,16.6,29.0,33.3,30.8


In [34]:
predict_data_29 = predict_data_29.rename(
    columns={
        "Opp_ADJO": "opp_ADJO",
        "Opp_ADJD": "opp_ADJD",
        "Opp_EFG_pct": "Opp_EFG_Pct",
    }
)

In [37]:
predict_data_process_29 = preprocess(predict_data_29)
predict_data_process_29 = predict_data_process_29.drop(
    columns=["date", "Team", "Opponent"]
)
predict_data_process_29 = predict_data_process_29.fillna(0)

# Check if 'Location_N' is missing in the prediction data
if "Location_N" not in predict_data_process_29.columns:
    predict_data_process_29["Location_N"] = 0  # Add the column and set all values to 0

predict_data_scaled_29 = scaler_X.transform(predict_data_process_29)

In [38]:
# predicting using normal NN

nn_predictions_scaled_29 = model.predict(predict_data_scaled_29)
nn_predictions_29 = scaler_y.inverse_transform(nn_predictions_scaled_29)

# Add predictions to the original dataset
predict_data_29["NN_Team_Score"] = nn_predictions_29[:, 0]
predict_data_29["NN_Opp_Score"] = nn_predictions_29[:, 1]

In [39]:
# predicting using MLP with Cross Validation

cv_predictions_scaled_29 = cv_model.predict(predict_data_scaled_29)
cv_predictions_29 = scaler_y.inverse_transform(cv_predictions_scaled_29)

# Add predictions to the original dataset
predict_data_29["CV_Team_Score"] = cv_predictions_29[:, 0]
predict_data_29["CV_Opp_Score"] = cv_predictions_29[:, 1]

In [40]:
# predicting using CV with Hyperparameter Tuning

hpt_predictions_scaled_29 = best_model.predict(predict_data_scaled_29)
hpt_predictions_29 = scaler_y.inverse_transform(hpt_predictions_scaled_29)

# Add predictions to the original dataset
predict_data_29["HPT_Team_Score"] = hpt_predictions_29[:, 0]
predict_data_29["HPT_Opp_Score"] = hpt_predictions_29[:, 1]

In [41]:
predict_data_29[
    [
        "Team",
        "Opponent",
        "NN_Team_Score",
        "NN_Opp_Score",
        "CV_Team_Score",
        "CV_Opp_Score",
        "HPT_Team_Score",
        "HPT_Opp_Score",
        # "Actual_Team",
        # "Actual_Opp",
    ]
]

,Team,Opponent,NN_Team_Score,NN_Opp_Score,CV_Team_Score,CV_Opp_Score,HPT_Team_Score,HPT_Opp_Score
0,Texas St.,UT Arlington,75.925911,76.683070,74.758713,73.185681,76.038748,74.534160
1,Washington,NJIT,78.789795,68.206457,76.282974,66.055808,74.031879,61.302444
2,North Carolina,Campbell,84.208435,71.456158,85.649064,72.851473,80.526861,65.755725
3,Iona,Harvard,68.820052,70.326079,71.612633,68.839300,66.058918,66.808499
4,Army,UTSA,77.613526,76.142965,71.995156,71.505234,72.782646,68.308180
5,Northwestern,Northeastern,75.659215,70.172706,72.941724,68.531063,76.236049,71.466229
6,Alabama,South Dakota St.,82.396979,75.015794,82.257407,71.630418,83.862504,73.745007
7,Ohio St.,Indiana St.,78.941534,73.942628,76.488892,70.979732,82.443069,77.440938
8,Illinois,Chicago St.,79.990492,60.174372,84.599585,62.345628,79.072349,55.041976
9,Oregon,Weber St.,81.279307,73.330775,79.804098,71.677491,80.628699,70.429103
